In [1]:
import numpy as np
import json
from keras.utils.np_utils import to_categorical
import time
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
with open('Random_Smart_Losses.json') as json_file:
    training = json.load(json_file)

In [3]:
for item in [training]:
    print(len(item['moves']))
    print(len(item['boards']))

54620
54620


In [4]:
X = np.array(training['boards'])
y = to_categorical(training['moves'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [5]:
for i in [X_train, X_test, X_val, y_train, y_test, y_val]:
    print(len(i))

49158
4369
1093
49158
4369
1093


In [6]:
from keras.models import load_model
parial_model = load_model('smart_model_partial.h5')

In [7]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics


history = parial_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

Train on 49158 samples, validate on 4369 samples
Epoch 1/10
49158/49158 [==============================] - 5s 97us/step - loss: 1.2494 - acc: 0.4822 - val_loss: 1.2840 - val_acc: 0.4722
Epoch 2/10
49158/49158 [==============================] - 5s 96us/step - loss: 1.2366 - acc: 0.4824 - val_loss: 1.2638 - val_acc: 0.4726
Epoch 3/10
49158/49158 [==============================] - ETA: 0s - loss: 1.2354 - acc: 0.481 - 5s 95us/step - loss: 1.2348 - acc: 0.4819 - val_loss: 1.2560 - val_acc: 0.4699
Epoch 4/10
49158/49158 [==============================] - 5s 101us/step - loss: 1.2343 - acc: 0.4826 - val_loss: 1.2583 - val_acc: 0.4797
Epoch 5/10
49158/49158 [==============================] - 5s 99us/step - loss: 1.2325 - acc: 0.4831 - val_loss: 1.2523 - val_acc: 0.4688
Epoch 6/10
49158/49158 [==============================] - 5s 100us/step - loss: 1.2317 - acc: 0.4839 - val_loss: 1.2511 - val_acc: 0.4715
Epoch 7/10
49158/49158 [==============================] - 5s 101us/step - loss: 1.2313 - 

In [8]:
parial_model.save('pre-trainied_loss_model.h5')

In [9]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

new_model = models.Sequential()

new_model.add(layers.Dense(729, activation = 'relu', input_shape = (9,)))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(9, activation = 'softmax'))



new_model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


# train the network
now = time.time()

history = new_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

print(time.time()-now)

print(new_model.evaluate(X_val, y_val))

Train on 49158 samples, validate on 4369 samples
Epoch 1/10
49158/49158 [==============================] - 5s 100us/step - loss: 1.4741 - acc: 0.4254 - val_loss: 1.6226 - val_acc: 0.4461
Epoch 2/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2731 - acc: 0.4766 - val_loss: 1.3379 - val_acc: 0.4660
Epoch 3/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2536 - acc: 0.4776 - val_loss: 1.3127 - val_acc: 0.4656
Epoch 4/10
49158/49158 [==============================] - 5s 97us/step - loss: 1.2446 - acc: 0.4811 - val_loss: 1.4824 - val_acc: 0.4395
Epoch 5/10
49158/49158 [==============================] - 5s 94us/step - loss: 1.2425 - acc: 0.4824 - val_loss: 1.2572 - val_acc: 0.4832
Epoch 6/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2384 - acc: 0.4820 - val_loss: 1.2527 - val_acc: 0.4722
Epoch 7/10
49158/49158 [==============================] - 5s 97us/step - loss: 1.2369 - acc: 0.4818 - val_loss: 1.2636 - val_acc

In [10]:
new_model_model.save('new_loss_model.h5')

NameError: name 'new_model_model' is not defined

In [ ]:
def make_prediction(model, board):
    prediction_list = list(model.predict(np.array([board]))[0])
    move = prediction_list.index(max(prediction_list))
    print(prediction_list)
    print(move)

In [ ]:
# Test 1
for model in [new_model, parial_model]:
    make_prediction(model, [0,0,0,
                            0,0,0,
                            0,0,0])
    print()

In [ ]:
# Test 2
print('correct answer is 6\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            1, 0, 0,
                            0,-1, 0])
    print()

In [ ]:
# Test 3
print('correct answer is 8\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            0, 1, 0,
                            0,-1, 0])
    print()

In [ ]:
# Test 4
print('correct answer is 5 or 6\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            1, 1, 0,
                            0,-1, -1])
    print()

In [ ]:
# Test 5
print('correct answer is 0\n')
for model in [new_model, parial_model]:
    make_prediction(model, [0,-1,  0,
                            1, 1, -1,
                            0,-1,  1])
    print()

In [ ]:
# Test 6
print('correct answer is 0\n')
for model in [new_model, parial_model]:
    make_prediction(model, [0,-1,  0,
                            1, 0, -1,
                            1,-1,  1])
    print()

In [ ]:
current_board = [[0,0,0],[0,0,0],[0,0,0]]

def board_to_prediction_list(board):
    vector_board = []
    for row in board:
        vector_board += row
    np_board = np.array([vector_board])

    return list(model.predict(np_board)[0])

def make_prediction(predicted_moves, available_moves):
    
    while True:
        move = predicted_moves.index(max(predicted_moves))
        predicted_moves[move] = 0

        if move in available_moves:
            return move

pred_list = board_to_prediction_list(current_board)
print(pred_list)
my_move = make_prediction(pred_list,set([3,5,6,7,8]))
print(my_move)

In [ ]:
predicted_moves.index(max(predicted_moves))

In [ ]:
model.save('smart_model.h5')